In [7]:
import pandas as pd
from snsynth import Synthesizer
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import plotly.graph_objects as go
from snsynth.mst import MSTSynthesizer

ModuleNotFoundError: No module named 'snsynth'

In [25]:
#aldult.data
pums = pd.read_csv("adult.data", header=None, names=[
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", 
    "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"])

print(pums.head())  

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50

In [85]:
# Pour epsilon = 1 avec allocation correcte
synth_1 = Synthesizer.create("mst", epsilon=0.99, verbose=True)
synth_1.fit(pums, preprocessor_eps=0.01)  # Allouer 0.1 au préprocesseur
pums_synth_1 = synth_1.sample(100)  # Allouer le reste au synthétiseur

Spent 0.01 epsilon on preprocessor, leaving 0.98 for training
Fitting with 8366677401600000 dimensions
Getting cliques
Estimating marginals


In [76]:
epsilon_total = 100

# Pour epsilon = 100 avec allocation correcte
synth = Synthesizer.create("mst", epsilon=epsilon_total, verbose=True)
synth.fit(pums, preprocessor_eps=1.0)  # Allouer 1.0 au préprocesseur
pums_synth = synth.sample(99)  # Allouer le reste au synthétiseur


Spent 1.0 epsilon on preprocessor, leaving 99.0 for training
Fitting with 8366677401600000 dimensions
Getting cliques
Estimating marginals


In [86]:
#print(pums_synth.head())

print(pums_synth_1.head())


   age     workclass    fnlwgt      education  education_num  \
0   41       Private  203161.6        HS-grad              9   
1   64   Federal-gov  137625.6        HS-grad              9   
2   47   Federal-gov  163840.0        HS-grad              9   
3   48       Private  255590.4      Assoc-voc             11   
4   30       Private  190054.4   Some-college             10   

        marital_status       occupation     relationship    race     sex  \
0            Separated    Other-service    Not-in-family   White  female   
1   Married-civ-spouse     Craft-repair   Other-relative   White  female   
2            Separated     Adm-clerical        Own-child   White    male   
3   Married-civ-spouse   Prof-specialty          Husband   White    male   
4   Married-civ-spouse    Other-service          Husband   White    male   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          0.05          0.05              65   United-States   <=50K  
1          0.0

In [87]:
print(pums.head())


   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race     sex  \
0        Never-married        Adm-clerical   Not-in-family   White    male   
1   Married-civ-spouse     Exec-managerial         Husband   White    male   
2             Divorced   Handlers-cleaners   Not-in-family   White    male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black    male   
4   Married-civ-spouse      Prof-specialty            Wife   Black  female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50K  
1 

In [30]:
# Afficher les valeurs uniques de la colonne 'sex'
print(pums['sex'].unique())


[' Male' ' Female']


In [88]:
# Suppression des espaces et mise en minuscules 
pums['sex'] = pums['sex'].str.strip().str.lower()
pums_synth['sex'] = pums_synth['sex'].str.strip().str.lower()
pums_synth_1['sex'] = pums_synth_1['sex'].str.strip().str.lower()


female_data = pums[pums['sex'] == 'female']
female_data_synth = pums_synth[pums_synth['sex'] == 'female']
female_data_synth_1 = pums_synth_1[pums_synth_1['sex'] == 'female']

male_data = pums[pums['sex'] == 'male']
male_data_synth = pums_synth[pums_synth['sex'] == 'male']
male_data_synth_1 = pums_synth_1[pums_synth_1['sex'] == 'male']

#print(female_data.head())
print(female_data_synth_1.head())


    age     workclass    fnlwgt      education  education_num  \
0    41       Private  203161.6        HS-grad              9   
1    64   Federal-gov  137625.6        HS-grad              9   
8    51       Private  137625.6   Some-college             10   
18   21   Federal-gov  137625.6   Some-college             10   
19   40       Private  255590.4      Bachelors             13   

         marital_status       occupation     relationship    race     sex  \
0             Separated    Other-service    Not-in-family   White  female   
1    Married-civ-spouse     Craft-repair   Other-relative   White  female   
8         Never-married    Other-service    Not-in-family   White  female   
18            Separated   Prof-specialty        Unmarried   White  female   
19             Divorced            Sales    Not-in-family   White  female   

    capital_gain  capital_loss  hours_per_week  native_country  income  
0           0.05          0.05              65   United-States   <=50K  


In [89]:
# La retraite à 60 ans
female_data = female_data[female_data['age'] <= 60]
female_data_synth = female_data_synth[female_data_synth['age'] <= 60]
female_data_synth_1 = female_data_synth_1[female_data_synth_1['age'] <= 60]


male_data = male_data[male_data['age'] <= 60]
male_data_synth = male_data_synth[male_data_synth['age'] <= 60]
male_data_synth_1 = male_data_synth_1[male_data_synth_1['age'] <= 60]



# Longueur des données pour les hommes et les femmes
print(len(female_data))
print(len(male_data))

10029
20200


In [90]:


# Calculer la moyenne des heures par semaine pour les femmes par tranche d'âge
women_avg_hours = female_data.groupby('age')['hours_per_week'].mean().reset_index()
women_avg_hours_synth = female_data_synth.groupby('age')['hours_per_week'].mean().reset_index()
women_avg_hours_synth_1 = female_data_synth_1.groupby('age')['hours_per_week'].mean().reset_index()

# Calculer la moyenne des revenus pour les hommes par tranche d'âge
men_avg_income = male_data.groupby('age')['hours_per_week'].mean().reset_index()
men_avg_income_synth = male_data_synth.groupby('age')['hours_per_week'].mean().reset_index()
men_avg_income_synth_1 = male_data_synth_1.groupby('age')['hours_per_week'].mean().reset_index()

#print(female_data.head())
#print(female_data_synth.head())

print(women_avg_hours.mean())
print('-------------------')
print(women_avg_hours_synth.mean())
print('----------------')
print(women_avg_hours_synth_1.mean())


age               38.500000
hours_per_week    37.302894
dtype: float64
-------------------
age               33.380952
hours_per_week    39.841270
dtype: float64
----------------
age               33.476190
hours_per_week    41.285714
dtype: float64


In [92]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer un subplot avec 2 lignes
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=("Heures par semaine des Femmes", "Revenu des Hommes"))

# Ajout des heures par semaine pour les femmes (rose)
fig.add_trace(go.Scatter(
    x=women_avg_hours['age'], 
    y=women_avg_hours['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaine (Femmes)',
    line=dict(color='lightpink'),  # Rose clair
    marker=dict(color='pink'),       # Rose
), row=1, col=1)

# Ajout des heures par semaine pour les femmes dans synth (rose)
fig.add_trace(go.Scatter(
    x=women_avg_hours_synth['age'], 
    y=women_avg_hours_synth['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaine (Femmes Synth)',
    line=dict(color='red'),          # Rose
    marker=dict(color='lightpink')   # Rose clair
), row=1, col=1)

# Ajout des heures par semaine pour les femmes dans synth 1000 (rose)

fig.add_trace(go.Scatter(
    x = women_avg_hours_synth_1['age'],
    y = women_avg_hours_synth_1['hours_per_week'],
    mode = 'lines+markers',
    name = 'Heures par semaine (Femmes Synth 100)',
    line = dict(color = 'orange'),
    marker = dict(color = 'orange')
), row = 1, col = 1)

# Ajout des revenus pour les hommes (bleu)
fig.add_trace(go.Scatter(
    x=men_avg_income['age'], 
    y=men_avg_income['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaines (Hommes)',
    line=dict(color='blue'),         # Bleu
    marker=dict(color='blue')   # Bleu clair
), row=2, col=1)

# Ajout des revenus pour les hommes dans synth (bleu clair)
fig.add_trace(go.Scatter(
    x=men_avg_income_synth['age'], 
    y=men_avg_income_synth['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaines (Hommes Synth)',
    line=dict(color='lightblue'),    # Bleu clair
    marker=dict(color='lightblue')         # Bleu
), row=2, col=1)

# Ajout des revenus pour les hommes dans synth 1000 (bleu clair)
fig.add_trace(go.Scatter(
    x=men_avg_income_synth_1['age'],
    y=men_avg_income_synth_1['hours_per_week'],
    mode='lines+markers',
    name='Heures par semaines (Hommes Synth 100)',
    line=dict(color='green'),
    marker=dict(color='green')
), row=2, col=1)


# Mise en forme du graphique
fig.update_layout(
    title="Comparaison des heures par semaine des Femmes et des Hommes selon l'Âge",
    xaxis_title="Âge",
    yaxis_title="Moyenne des heures par semaine",
    legend_title="Variable",
    template="plotly_white"
)

# Affichage du graphique
fig.show()


In [96]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer un subplot avec 2 lignes et 1 colonne pour les histogrammes
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=("Distribution des Heures par semaine des Femmes", 
                                    "Distribution des Heures par semaine des Hommes"))

# Ajout de l'histogramme des heures par semaine pour les femmes (rose)
fig.add_trace(go.Histogram(
    x=women_avg_hours['hours_per_week'],  # Données des femmes
    name='Heures par semaine (Femmes)',
    marker_color='lightpink',  # Couleur rose
    opacity=0.7
), row=1, col=1)

# Ajout de l'histogramme des heures par semaine pour les femmes synthétiques (rouge)
fig.add_trace(go.Histogram(
    x=women_avg_hours_synth['hours_per_week'],  # Données synthétiques des femmes
    name='Heures par semaine (Femmes Synth)',
    marker_color='red',  # Couleur rouge
    opacity=0.5
), row=1, col=1)

# Ajout de l'histogramme des heures par semaine pour les femmes synthétiques 1000 (orange)
fig.add_trace(go.Histogram(
    x=women_avg_hours_synth_1['hours_per_week'],  # Données synthétiques 1000 des femmes
    name='Heures par semaine (Femmes Synth 100)',
    marker_color='orange',  # Couleur orange
    opacity=0.5
), row=1, col=1)

# Ajout de l'histogramme des heures par semaine pour les hommes (bleu)
fig.add_trace(go.Histogram(
    x=men_avg_income['hours_per_week'],  # Données des hommes
    name='Heures par semaine (Hommes)',
    marker_color='blue',  # Couleur bleue
    opacity=0.7
), row=2, col=1)

# Ajout de l'histogramme des heures par semaine pour les hommes synthétiques (bleu clair)
fig.add_trace(go.Histogram(
    x=men_avg_income_synth['hours_per_week'],  # Données synthétiques des hommes
    name='Heures par semaine (Hommes Synth)',
    marker_color='lightblue',  # Couleur bleu clair
    opacity=0.5
), row=2, col=1)

# Ajout de l'histogramme des heures par semaine pour les hommes synthétiques 1000 (vert)
fig.add_trace(go.Histogram(
    x=men_avg_income_synth_1['hours_per_week'],  # Données synthétiques 1000 des hommes
    name='Heures par semaine (Hommes Synth 100)',
    marker_color='green',  # Couleur verte
    opacity=0.5
), row=2, col=1)

# Mise en forme du graphique
fig.update_layout(
    title="Distribution des Heures par semaine selon le Sexe",
    xaxis_title="Ages des individus",
    yaxis_title="Nombre d'heures de travail moyen par semaine",
    barmode='overlay',  # Superposer les histogrammes
    template="plotly_white"
)

# Affichage du graphique
fig.show()
